In [1]:
using GraphPPL, ReactiveMP, Distributions
# using Plots

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1662


In [44]:
function f(x)
    return x
end

function f_inv(x)
    return x
end

c = randn(2);

@model function NKS()
    y2 = datavar(Float64)
    c = zeros(2); c[1] = 1.0;

    # γ ~ Gamma(α=1.0, β=1.0)
    x ~ MvNormal(μ=zeros(2), Λ=diageye(2))
    z ~ f(x) where {meta=ET(f_inv)}
    y1 ~ Normal(μ=dot(z, c), σ²=1.0)
    y2 ~ Normal(μ=y1, σ²=0.5)
end

In [45]:
# constraints = @constraints begin
#     q(x, γ) = q(x)q(γ)
# end

imarginals = (x = MvNormal(zeros(2), diageye(2)), γ=Gamma(1.0, 1.0))

result = inference(model = Model(NKS), data=(y2=1.0,), free_energy=false)

"hello" = "hello"
g(x_hat) = 0.0


LoadError: DimensionMismatch: jacobian(f, x) expects that f(x) is an array. Perhaps you meant gradient(f, x)?

In [35]:
collectStatistics(MvNormalMeanCovariance(zeros(2), diageye(2)))

LoadError: type MvNormalMeanCovariance has no field dist